In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import json

# df = pd.read_parquet("../part-00000-8bcd0748-094a-4de0-8574-2328d1833c16-c000.gz.parquet", engine='pyarrow')
# df = pd.read_parquet("../part-00000-ae0248c1-9d9a-4943-8350-71a5267f9c51-c000.gz.parquet", engine='pyarrow')
# df = pd.read_parquet("../part-00000-96ac3cdd-9fe4-4189-a066-f7b73d199c16-c000.gz.parquet", engine='pyarrow')
df = pd.read_parquet("../part-00000-8ef97ec5-2cb7-4b7a-8a07-0935e967ee8e-c000.gz.parquet", engine='pyarrow')
df
# print("done")

,key,surtUrl,timestamp,originalUrl,mime,status,digest,redirectUrl,meta,contentLength,offset,filename,allheader,payload
0,"http://(com,visitcalifornia,www,)/node/16026/m...","http://(com,visitcalifornia,www,)/node/16026/m...",20161224,http://www.visitcalifornia.com/node/16026/more...,text/html,200,-,-,-,81764,363,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Sat, 24 Dec 2016 10:0..."
1,"http://(com,visitcalifornia,www,)/node/16026/m...","http://(com,visitcalifornia,www,)/node/16026/m...",20170926,http://www.visitcalifornia.com/node/16026/more...,text/html,200,-,-,-,85985,14805,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Tue, 26 Sep 2017 05:0..."
2,"http://(com,visitcalifornia,www,)/node/16026/m...","http://(com,visitcalifornia,www,)/node/16026/m...",20161226,http://www.visitcalifornia.com/node/16026/more...,text/html,200,-,-,-,81216,29968,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Mon, 26 Dec 2016 10:2..."
3,"http://(com,visitcalifornia,www,)/node/16026/m...","http://(com,visitcalifornia,www,)/node/16026/m...",20161226,http://www.visitcalifornia.com/node/16026/more...,text/html,200,-,-,-,81509,44347,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Mon, 26 Dec 2016 10:2..."
4,"http://(com,visitcalifornia,www,)/node/16026/m...","http://(com,visitcalifornia,www,)/node/16026/m...",20161226,http://www.visitcalifornia.com/node/16026/more...,text/html,200,-,-,-,81650,58753,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.1 200 OK\r\nDate: Mon, 26 Dec 2016 04:3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48804,"http://(com,visitcalifornia,yearinreview,)/doc...","http://(com,visitcalifornia,yearinreview,)/doc...",20161202,http://yearinreview.visitcalifornia.com/docs/t...,text/html,404,-,-,-,51408,818558951,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,"HTTP/1.0 404 Not Found\r\nDate: Fri, 02 Dec 20..."
48805,"http://(com,visitcalifornia,yearinreview,)/fav...","http://(com,visitcalifornia,yearinreview,)/fav...",20150907,http://yearinreview.visitcalifornia.com/favico...,text/html,404,-,-,-,1813,818578807,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,HTTP/1.1 404 Not Found\r\nContent-Length: 1635...
48806,"https://(com,visitcalifornia,yearinreview,)/ro...","https://(com,visitcalifornia,yearinreview,)/ro...",20170817,https://yearinreview.visitcalifornia.com/robot...,text/html,404,-,-,-,336,818580081,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,HTTP/1.1 404 Not Found\r\nServer: nginx/1.10.3...
48807,"https://(com,visitcalifornia,yearinreview,)/ro...","https://(com,visitcalifornia,yearinreview,)/ro...",20180519,https://yearinreview.visitcalifornia.com/robot...,text/html,404,-,-,-,336,818580560,California2_www-visitcalifornia-com_200103-201...,file:/home/xw0078/data/tourism-data/vt-tourism...,HTTP/1.1 404 Not Found\r\nServer: nginx/1.10.3...


In [2]:
df.timestamp[0]

'20161224'

In [3]:
# df['timestamp'] = pd.to_datetime(df['timestamp'], infer_datetime_format=True)

In [4]:
def headerAndTime(df):
    #list that will hold information for every row 
    tableInfo = []
    #list that will hold indices of rows with errors
    errors = []
    for row in range(df.shape[0]):
        try:
            #dictionary to store information from the row
            quickDict = {}
            #grabbing information from each column of the row
            quickDict["Row"] = row
            soup = BeautifulSoup(df.payload[row], 'html.parser')
            quickDict["Header"] = soup.title.text
            quickDict["Timestamp"] = df.timestamp[row]
            quickDict["URL"] = df.originalUrl[row]
            tableInfo.append(quickDict)
        except AttributeError:
            #some rows don't have an HTML file in the payload
            #when trying to use soup.title.text above, it will give an AttributeError because it has no title.text attribute
            #this exception adds the row index to a list titled "errors" to be able to see which rows had the error in case it's needed
            errors.append(row)
    return tableInfo, errors

In [5]:
tableInfo, errors = headerAndTime(df)

In [6]:
print(len(errors))

7888


In [7]:
import csv
csv_columns = ['Row','Header','Timestamp', 'URL']
csv_file = "headerTimestampUrl4.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in tableInfo:
            writer.writerow(data)
except IOError:
    print("I/O error")